In [1]:
!pip install complextorch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 2.1 MB/s eta 0:00:00


In [2]:
import complextorch
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import complextorch as comptorch
import complextorch.nn as compnn
from torch.utils.data import DataLoader, random_split
from tqdm import tqdm

In [3]:
batch_size = 100
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])
train_set = datasets.CIFAR10('./data', train=True, transform=trans, download=True)
# train_set = complextorch.CVTensor(train_set, i=torch.zeros_like((train_set)))
test_set = datasets.CIFAR10('./data', train=False, transform=trans, download=True)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=True)

100%|██████████| 170498071/170498071 [00:18<00:00, 9289254.33it/s] 


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [5]:
class ComplexNet(nn.Module):
    def __init__(self):
        super(ComplexNet, self).__init__()
        self.conv1 = compnn.CVConv2d(3, 10, kernel_size=(5, 5), stride=(1, 1))
        self.relu1 = compnn.CVCardiod()
        self.conv2 = compnn.CVConv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
        self.fc1 = compnn.CVLinear(5 * 5 * 20, 50)
        self.fc2 = compnn.CVLinear(50, 10)
        self.smx = compnn.PhaseSoftMax(dim=1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        mxpool1 = compnn.CVAdaptiveAvgPool2d((x.shape[2] // 2, x.shape[3] // 2))
        x = mxpool1(x)
        x = self.conv2(x)
        x = self.relu1(x)
        mxpool1 = compnn.CVAdaptiveAvgPool2d((x.shape[2] // 2, x.shape[3] // 2))
        x = mxpool1(x)
        x = x.view(-1, 5 * 5 * 20)
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = x.abs()
        out = self.smx(x)
        return out

In [6]:
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    total_correct = 0
    total_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = complextorch.CVTensor(data, i=torch.zeros_like(data)).to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        _, pred = torch.max(output, 1)
        # loss = compnn.CVCauchyError()
        # loss = loss_fun(output, target)
        # target_oh = to_categorical(target, 10)
        loss = F.cross_entropy(output, target)
        loss.backward()
        optimizer.step()
        total_correct += torch.sum(pred == target.data).item()
        total_loss += loss.item()
        if batch_idx % 100 == 0 and batch_idx > 0:
            print('Train Epoch: {:3} [{:6}/{:6} ({:3.0f}%)]\tLoss: {:.6f}\t Acc:{:.6f}'.format(
                epoch,
                batch_idx * len(data),
                len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                total_loss / (batch_idx + 1),
                total_correct/((batch_idx + 1) * batch_size)
            )
            )

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')
model = ComplexNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [8]:
for epoch in tqdm(range(50)):
    train(model, device, train_loader, optimizer, epoch)

  0%|          | 0/50 [00:00<?, ?it/s]

Train Epoch:   0 [ 10000/ 50000 ( 20%)]	Loss: 2.212387	 Acc:0.238713
Train Epoch:   0 [ 20000/ 50000 ( 40%)]	Loss: 2.175684	 Acc:0.277313
Train Epoch:   0 [ 30000/ 50000 ( 60%)]	Loss: 2.152604	 Acc:0.301163
Train Epoch:   0 [ 40000/ 50000 ( 80%)]	Loss: 2.131905	 Acc:0.323267


  2%|▏         | 1/50 [00:18<15:21, 18.81s/it]

Train Epoch:   1 [ 10000/ 50000 ( 20%)]	Loss: 2.045575	 Acc:0.411782
Train Epoch:   1 [ 20000/ 50000 ( 40%)]	Loss: 2.031378	 Acc:0.427015
Train Epoch:   1 [ 30000/ 50000 ( 60%)]	Loss: 2.026704	 Acc:0.431429
Train Epoch:   1 [ 40000/ 50000 ( 80%)]	Loss: 2.022033	 Acc:0.436434


  4%|▍         | 2/50 [00:35<14:09, 17.70s/it]

...

 96%|█████████▌| 48/50 [13:18<00:33, 16.66s/it]

Train Epoch:  48 [ 10000/ 50000 ( 20%)]	Loss: 1.664257	 Acc:0.798119
Train Epoch:  48 [ 20000/ 50000 ( 40%)]	Loss: 1.669792	 Acc:0.792189
Train Epoch:  48 [ 30000/ 50000 ( 60%)]	Loss: 1.670573	 Acc:0.791728
Train Epoch:  48 [ 40000/ 50000 ( 80%)]	Loss: 1.674323	 Acc:0.787955


 98%|█████████▊| 49/50 [13:34<00:16, 16.58s/it]

Train Epoch:  49 [ 10000/ 50000 ( 20%)]	Loss: 1.663043	 Acc:0.800396
Train Epoch:  49 [ 20000/ 50000 ( 40%)]	Loss: 1.665654	 Acc:0.797313
Train Epoch:  49 [ 30000/ 50000 ( 60%)]	Loss: 1.668057	 Acc:0.794352
Train Epoch:  49 [ 40000/ 50000 ( 80%)]	Loss: 1.668841	 Acc:0.793716


100%|██████████| 50/50 [13:51<00:00, 16.63s/it]


In [9]:
model.eval()

total_correct = 0

with torch.no_grad():

    for batch_idx, (data, label) in enumerate(test_loader):

        data, label = complextorch.CVTensor(data, i=torch.zeros_like(data)).to(device), label.to(device)

        out = model(data)

        _, pred = torch.max(out, 1)

        total_correct += (label == pred).sum().item()

    print(100 * total_correct/len(test_loader.dataset))

torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([100, 3, 32, 32])
torch.Size([10

## Tensorflow implementation


In [10]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip install cvnn
!pip install tensorflow_addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for cvnn: filename=cvnn-2.0-py2.py3-none-any.whl size=47610 sha256=1b3e5a5f71665987654d19d65f6339240322345d9fa530677ea79c858ec4a477
  Stored in directory: /root/.cache/pip/wheels/91/7c/d5/a4816f12ef5f955ed83cd22822d376f57aa0bc00b7ea2c4486
Successfully built cvnn
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 9.9 MB/s eta 0:00:00


In [11]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import cvnn.layers as complex_layers
from cvnn.losses import ComplexAverageCrossEntropy
from cvnn.metrics import ComplexCategoricalAccuracy
import numpy as np
from pdb import set_trace
from keras.utils import to_categorical
import warnings

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


# for complex input

In [4]:
def to_categorical(y, num_classes):
    """ 1-hot encodes a tensor """
    return torch.eye(num_classes)[y]

In [29]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images.astype(dtype=np.float32) / 255.0, test_images.astype(dtype=np.float32) / 255.0
train_labels = to_categorical(train_labels, 10)
test_labels = to_categorical(test_labels, 10)

In [14]:
def own_complex_fit(epochs=10):
    tf.random.set_seed(1)
    init = tf.keras.initializers.GlorotUniform(seed=117)
    model = models.Sequential()
    model.add(complex_layers.ComplexInput(input_shape=(32, 32, 3)))
    model.add(complex_layers.ComplexConv2D(10, (5, 5), activation='complex_cardioid', input_shape=(32, 32, 3)))
    model.add(complex_layers.ComplexMaxPooling2D((2, 2)))
    model.add(complex_layers.ComplexBatchNormalization())
    model.add(complex_layers.ComplexConv2D(20, (5, 5), activation='complex_cardioid', kernel_initializer=init))
    model.add(complex_layers.ComplexMaxPooling2D((2, 2)))
    # model.add(complex_layers.ComplexConv2D(64, (3, 3), activation='complex_cardioid', kernel_initializer=init))
    model.add(complex_layers.ComplexFlatten())
    model.add(complex_layers.ComplexDense(50, activation='complex_cardioid', kernel_initializer=init,))
    model.add(complex_layers.ComplexDense(10, activation='cart_softmax'))
    # model.summary()
    model.compile(optimizer='adam',
                  loss=ComplexAverageCrossEntropy(),
                  metrics=ComplexCategoricalAccuracy())
    weigths = model.get_weights()
    with tf.GradientTape() as tape:
        loss = model.compiled_loss(y_true=tf.convert_to_tensor(test_labels), y_pred=model(test_images))
        gradients = tape.gradient(loss, model.trainable_weights)  # back-propagation
    history = model.fit(train_images, train_labels, epochs=epochs, validation_data=(test_images, test_labels))
    test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
    logs = {
        'weights_at_init': weigths,
        'loss': loss,
        'gradients': gradients,
        'weights_at_end': model.get_weights()
    }
    return history, logs

In [15]:
own, own_logs = own_complex_fit(epochs=50)

Training was None and now is 0


Epoch 1/50


1563/1563 [==============================] - 36s 19ms/step - loss: 1.4583 - complex_categorical_accuracy: 0.4948 - val_loss: 1.5401 - val_complex_categorical_accuracy: 0.4858
Epoch 2/50
1563/1563 [==============================] - 30s 19ms/step - loss: 1.0939 - complex_categorical_accuracy: 0.6282 - val_loss: 1.6242 - val_complex_categorical_accuracy: 0.5168
Epoch 3/50
1563/1563 [==============================] - 30s 19ms/step - loss: 0.9700 - complex_categorical_accuracy: 0.6711 - val_loss: 0.9993 - val_complex_categorical_accuracy: 0.6577
Epoch 4/50
1563/1563 [==============================] - 30s 19ms/step - loss: 0.8860 - complex_categorical_accuracy: 0.7011 - val_loss: 1.2348 - val_complex_categorical_accuracy: 0.5938
Epoch 5/50
1563/1563 [==============================] - 29s 19ms/step - loss: 0.8266 - complex_categorical_accuracy: 0.7220 - val_loss: 1.1968 - val_complex_categorical_accuracy: 0.6133
Epoch 6/50
1563/1563 [==============================] - 29s 19ms/step - loss: 0.7

# for real input

In [32]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images.astype(dtype=np.float32) / 255.0, test_images.astype(dtype=np.float32) / 255.0

In [33]:
def keras_fit(epochs=10, use_bias=True):
    tf.random.set_seed(1)
    init = tf.keras.initializers.GlorotUniform(seed=117)
    model = models.Sequential()
    model.add(layers.Conv2D(10, (5, 5), activation='relu', input_shape=(32, 32, 3), kernel_initializer=init,
                            use_bias=use_bias))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(20, (5, 5), activation='relu', kernel_initializer=init, use_bias=use_bias))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(50, activation='relu', kernel_initializer=init, use_bias=use_bias))
    model.add(layers.Dense(10, kernel_initializer=init, use_bias=use_bias, activation='softmax'))
    model.compile(optimizer='adam',
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                  metrics=['accuracy'])
    weigths = model.get_weights()
    with tf.GradientTape() as tape:
        # for elem, label in iter(ds_train):
        loss = model.compiled_loss(y_true=tf.convert_to_tensor(test_labels), y_pred=model(test_images))
        gradients = tape.gradient(loss, model.trainable_weights)  # back-propagation
    history = model.fit(train_images, train_labels, epochs=epochs, validation_data=(test_images, test_labels))
    test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
    logs = {
        'weights_at_init': weigths,
        'loss': loss,
        'gradients': gradients,
        'weights_at_end': model.get_weights()
    }
    return history, logs

In [34]:
keras, keras_logs = keras_fit(epochs=50, use_bias=False)

Epoch 1/50
1563/1563 [==============================] - 10s 6ms/step - loss: 1.5391 - accuracy: 0.4413 - val_loss: 1.4017 - val_accuracy: 0.4921
Epoch 2/50
1563/1563 [==============================] - 8s 5ms/step - loss: 1.2555 - accuracy: 0.5526 - val_loss: 1.2401 - val_accuracy: 0.5517
Epoch 3/50
1563/1563 [==============================] - 8s 5ms/step - loss: 1.1477 - accuracy: 0.5912 - val_loss: 1.1929 - val_accuracy: 0.5829
Epoch 4/50
1563/1563 [==============================] - 8s 5ms/step - loss: 1.0790 - accuracy: 0.6168 - val_loss: 1.2815 - val_accuracy: 0.5535
Epoch 5/50
1563/1563 [==============================] - 9s 5ms/step - loss: 1.0288 - accuracy: 0.6376 - val_loss: 1.2301 - val_accuracy: 0.5786
Epoch 6/50
1563/1563 [==============================] - 8s 5ms/step - loss: 0.9848 - accuracy: 0.6536 - val_loss: 1.0807 - val_accuracy: 0.6170
Epoch 7/50
1563/1563 [==============================] - 7s 5ms/step - loss: 0.9534 - accuracy: 0.6631 - val_loss: 1.0870 - val_accuracy